<a href="https://colab.research.google.com/github/junp89/Desafio_DIO/blob/main/Desafio_de_Projeto_Santander_Bootcamp_2023_Ci%C3%AAncia_de_Dados_com_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Explorando IA Generativa em um Pipeline de ETL com Python

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.



In [12]:

# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

import pandas as pd
import requests
import json

## **E**xtract

Extração da lista de IDs de usuário a partir do arquivo CSV.

In [13]:
df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[5462, 5465, 5466]


Para cada ID, foi feita uma requisição GET para obter os dados do usuário correspondente.

In [14]:
def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 5462,
    "name": "Joana Silva",
    "account": {
      "id": 5794,
      "number": "00089-9",
      "agency": "0001",
      "balance": 0.0,
      "limit": 5000.0
    },
    "card": {
      "id": 5304,
      "number": "**** **** **** 9391",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 5465,
    "name": "George Ramos",
    "account": {
      "id": 5797,
      "number": "00089-2",
      "agency": "0001",
      "balance": 0.0,
      "limit": 2000.0
    },
    "card": {
      "id": 5307,
      "number": "**** **** **** 9751",
      "limit": 5000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 5466,
    "name": "Rosa Santos",
    "account": {
      "id": 5798,
      "number": "00089-3",
      "agency": "0001",
      "balance": 0.0,
      "limit": 7000.0
    },
    "card": {
      "id": 5308,
      "number": "**** **** **** 9764",
      "limit": 20000.0
    },
    "features": [],
    "news": []
  }
]


## **T**ransform

Utilizando a API do OpenAI GPT-3.5 foi gerada uma mensagem de marketing personalizada para cada usuário sobre a importância dos investimentos financeiros.

In [ ]:
!pip install openai

In [15]:
# API Key da OpenAI
openai_api_key = 'OPENAI_API_KEY'

In [18]:
import openai

openai.api_key = openai_api_key

def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em markting bancário."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem personalizada criativa destinada a {user['name']} que transmita a importância dos investimentos. (máximo de 95 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

Joana, invista no seu futuro financeiro e transforme seus sonhos em realidade! 💰✨
George, transforme seu futuro: invista agora!
Rosa, garanta seu futuro com investimentos inteligentes!


## **L**oad

Por fim, a lista "news" de cada usuário foi atualizada na API com a mensagem gerada.

In [19]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Joana Silva updated? True!
User George Ramos updated? True!
User Rosa Santos updated? True!
